# 进程和线程

## 多进程
- Unix / Linux 下，fock() 实现多进程
- 由于 windows 无 fock(), 所以若要实现跨平台多进程，可以使用 multiprocessing 模块
- 进程间通信通过 Queue, Pipes 等实现

### 多进程 + 池

In [ ]:
import os
from python_utils import long_time_task
from multiprocessing import Pool

if __name__ == '__main__':
    print(f'Main process {os.getpid()}')
    p = Pool(4)
    for i in range(8):
        p.apply_async(long_time_task, args=(i,))
    print('Waiting for all subprocesses done ...')
    p.close()
    p.join()
    print('All subprocesses done')


Main process 32825
Waiting for all subprocesses done ...
Run task 0(pid:33700) ...
Run task 1(pid:33701) ...
Run task 2(pid:33703) ...
Run task 3(pid:33702) ...
Finish 3(pid:33702) task with 0.1 ...
Run task 4(pid:33702) ...
Finish 2(pid:33703) task with 0.3 ...
Run task 5(pid:33703) ...
Finish 5(pid:33703) task with 1.0 ...
Run task 6(pid:33703) ...
Finish 4(pid:33702) task with 1.4 ...
Run task 7(pid:33702) ...
Finish 7(pid:33702) task with 0.1 ...
Finish 1(pid:33701) task with 1.8 ...
Finish 0(pid:33700) task with 2.3 ...
Finish 6(pid:33703) task with 2.0 ...
All subprocesses done


### 子进程 subprocess
很多时候，子进程并不一定是自身，而是一个外部进程，如 subprocess 实现 shell 调用

In [4]:
import subprocess

print('$ nslookup www.python.org')
r = subprocess.call(['nslookup', 'www.python.org'])
print(f'Exit code:{r}')

$ nslookup www.python.org
Server:		116.228.111.118
Address:	116.228.111.118#53

Non-authoritative answer:
www.python.org	canonical name = dualstack.python.map.fastly.net.
Name:	dualstack.python.map.fastly.net
Address: 151.101.76.223

Exit code:0


### 子进程输入传值 communicate

In [5]:
import subprocess


"""
Last login: Wed Nov 20 18:46:35 on ttys000
limengjie@limengjiedeMacBook-Pro ~ % nslookup
> set q=mx
> python.org
Server:		116.228.111.118
Address:	116.228.111.118#53

Non-authoritative answer:
python.org	mail exchanger = 50 mail.python.org.

Authoritative answers can be found from:
> exit
"""

print('$ nslookup')
p = subprocess.Popen(
    ['nslookup'],
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)
output, err = p.communicate(b'set q=mx\npython.org\nexit\n')
print(output.decode('utf-8'))
print(f'Exit code:{p.returncode}')

$ nslookup
Server:		116.228.111.118
Address:	116.228.111.118#53

Non-authoritative answer:
python.org	mail exchanger = 50 mail.python.org.

Authoritative answers can be found from:


Exit code:0


### 进程间通信
Queue 实现进程间交换数据

In [ ]:
from python_utils import write_func, read_func
from multiprocessing import Process, Queue


if __name__ == '__main__':
    q = Queue()
    pw = Process(target=write_func, args=(q,))
    pr = Process(target=read_func, args=(q,))
    pw.start()
    pr.start()
    pw.join()
    pr.terminate()






Process 33707 to read:Process 33706 to write:

Put A to queue ...
Get A from queue
Put B to queue ...
Get B from queue
Put C to queue ...
Get C from queue


# 多线程
多任务，可以由多进程完成，可以由一个进程内的多线程完成。
- threading 封装了 \_thread 模块
- Python 线程是真正的 Posix Thread，而不是模拟出来的线程
- 线程是操作系统直接支持的执行单元
- 启动一个函数传入并创建 Thread 实例，调用 start() 开始执行

In [8]:
import time, threading

def loop():
    print(f'thread {threading.current_thread().name} is running ...')
    n = 0
    while n < 5:
        n += 1
        print(f'thread {threading.current_thread().name} >> {n}')
        time.sleep(2)
    print(f'thread {threading.current_thread().name} ended!')

print(f'thread {threading.current_thread().name} is running ...')
t = threading.Thread(target=loop, name='LoopThread')
t.start()
t.join()
print(f'thread {threading.current_thread().name} ended!')

thread MainThread is running ...
thread LoopThread is running ...
thread LoopThread >> 1
thread LoopThread >> 2
thread LoopThread >> 3
thread LoopThread >> 4
thread LoopThread >> 5
thread LoopThread ended!
thread MainThread ended!


## Lock
- 多线程所有变量共享，若同时修改一个不变量，会有风险改乱
- 确保某段关键代码只能有一个线程从头到尾完整执行
- 多个锁的存爱，易出现死锁
- GIL（100条指令轮换） 导致 python 无法利用多核，不过 python 的多进程才能避免 GIL 的影响

In [9]:
import time, threading

balance = 0
lock = threading.Lock()

def change_it(n):
    global balance
    balance = balance + n
    balance = balance - n

def run_thread(n):
    for _ in range(1000000):
        lock.acquire()
        try:
            change_it(n)
        finally:
            lock.release()

t1 = threading.Thread(target=run_thread, args=(2**5,))
t2 = threading.Thread(target=run_thread, args=(2**10,))
t1.start()
t2.start()
t1.join()
t2.join()
print(balance)

0


## ThreadLocal
解决参数在一个线程中的各个功能函数之间互相传递的问题。如：为每个线程绑定一个 DB connection,http request, useraccount/RBAC


In [10]:
import threading

local_school = threading.local()

def process_thread(name):
    local_school.student = name
    process_student()

def process_student():
    std = local_school.student
    print(f'Hello {std} in {threading.current_thread().name}')

t1 = threading.Thread(target=process_thread, args=('Alice',), name='Thread-A')
t2 = threading.Thread(target=process_thread, args=('Bob',), name='Thread-B')
t1.start()
t2.start()
t1.join()
t2.join()


Hello Alice in Thread-A
Hello Bob in Thread-B
